In [0]:
# COMMAND ----------
# ===================================================================
# SEC SMART MONEY - GOLD LAYER
# Production-Grade Analytics & Aggregations (VIEWS)
# ===================================================================

from datetime import datetime, timedelta
import traceback

print("""
╔═════════════════════════════════════════════════════════════════════════════╗
║                                                                             ║
║                    🥇 SEC SMART MONEY - GOLD LAYER                         ║
║                                                                             ║
║        Creating analytical views and business intelligence layer             ║
║                                                                             ║
╚═════════════════════════════════════════════════════════════════════════════╝
""")

# COMMAND ----------
# ===================================================================
# PARAMETERS - Production Configuration
# ===================================================================

# Define parameters with clear labels
dbutils.widgets.text(
    "catalog_name",
    "fintech_analytics",
    "1. Catalog Name"
)

dbutils.widgets.dropdown(
    "environment",
    "prod",
    ["dev", "staging", "prod"],
    "2. Environment"
)

dbutils.widgets.text(
    "min_conviction_score",
    "50",
    "3. Minimum Conviction Score"
)

dbutils.widgets.text(
    "lookback_days",
    "365",
    "4. Lookback Days"
)

# COMMAND ----------
# ===================================================================
# GET PARAMETER VALUES - Safe Retrieval Pattern
# ===================================================================

try:
    catalog_name = dbutils.widgets.get("catalog_name")
    if not catalog_name:
        catalog_name = "fintech_analytics"
except:
    catalog_name = "fintech_analytics"

try:
    environment = dbutils.widgets.get("environment")
    if not environment:
        environment = "prod"
except:
    environment = "prod"

try:
    min_conviction = int(dbutils.widgets.get("min_conviction_score"))
    if min_conviction < 0:
        min_conviction = 50
except:
    min_conviction = 50

try:
    lookback_days = int(dbutils.widgets.get("lookback_days"))
    if lookback_days <= 0:
        lookback_days = 365
except:
    lookback_days = 365

# Construct schemas
silver_schema = f"{catalog_name}.silver"
gold_schema = f"{catalog_name}.gold"

# Calculate dates
run_date = datetime.now().strftime("%Y-%m-%d")
lookback_start = (datetime.now() - timedelta(days=lookback_days)).strftime("%Y-%m-%d")

# Print configuration
print(f"""
📋 EXECUTION PARAMETERS:
  Catalog Name:          {catalog_name}
  Silver Schema:         {silver_schema}
  Gold Schema:           {gold_schema}
  Environment:           {environment}
  Minimum Conviction:    {min_conviction}
  Lookback Days:         {lookback_days}
  Lookback Start:        {lookback_start}
  Run Date:              {run_date}
""")


╔═════════════════════════════════════════════════════════════════════════════╗
║                                                                             ║
║                    🥇 SEC SMART MONEY - GOLD LAYER                         ║
║                                                                             ║
║        Creating analytical views and business intelligence layer             ║
║                                                                             ║
╚═════════════════════════════════════════════════════════════════════════════╝


📋 EXECUTION PARAMETERS:
  Catalog Name:          fintech_analytics
  Silver Schema:         fintech_analytics.silver
  Gold Schema:           fintech_analytics.gold
  Environment:           prod
  Minimum Conviction:    50
  Lookback Days:         365
  Lookback Start:        2025-02-22
  Run Date:              2026-02-22



In [0]:
# ===================================================================
# STEP 1: Verify Silver Layer Exists
# ===================================================================

print("\n" + "="*80)
print("STEP 1: Verifying Silver Layer")
print("="*80)

try:
    silver_count = spark.sql(f"""
    SELECT COUNT(*) as count
    FROM {silver_schema}.silver_fact_insider_transactions
    """).collect()[0]['count']
    
    print(f"✅ Silver layer verified: {silver_count:,} rows available")
    
except Exception as e:
    print(f"❌ Error accessing silver layer: {str(e)}")
    raise


STEP 1: Verifying Silver Layer
✅ Silver layer verified: 394 rows available


In [0]:
# ===================================================================
# STEP 2: CREATE GOLD VIEWS
# ===================================================================

print("\n" + "="*80)
print("STEP 2: Creating Gold Views")
print("="*80)

# COMMAND ----------
# VIEW 1: gold_insider_summary_by_company
# Summarizes insider activity by company

try:
    print("\n📊 Creating VIEW: gold_insider_summary_by_company")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_insider_summary_by_company AS
    SELECT 
        company_name,
        COUNT(DISTINCT insider_name) as num_insiders,
        COUNT(*) as total_transactions,
        SUM(CASE WHEN acquired_disposed = 'A' THEN 1 ELSE 0 END) as total_buys,
        SUM(CASE WHEN acquired_disposed = 'D' THEN 1 ELSE 0 END) as total_sells,
        ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) as avg_confidence,
        MAX(filing_date) as latest_filing_date
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    GROUP BY company_name
    ORDER BY total_transactions DESC
    """)
    
    view_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_insider_summary_by_company").collect()[0]['count']
    print(f"    ✅ Created VIEW: {view_count:,} company records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise


STEP 2: Creating Gold Views

📊 Creating VIEW: gold_insider_summary_by_company
    ✅ Created VIEW: 26 company records


In [0]:
# COMMAND ----------
# VIEW 2: gold_high_activity_insiders
# Identifies insiders with high transaction activity

try:
    print("\n📊 Creating VIEW: gold_high_activity_insiders")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_high_activity_insiders AS
    SELECT 
        insider_name,
        COUNT(DISTINCT company_name) as num_companies,
        COUNT(*) as total_transactions,
        ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) as avg_confidence,
        MAX(filing_date) as latest_transaction
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    GROUP BY insider_name
    HAVING COUNT(*) >= 5
    ORDER BY total_transactions DESC
    """)
    
    insider_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_high_activity_insiders").collect()[0]['count']
    print(f"    ✅ Created VIEW: {insider_count:,} insider records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise



📊 Creating VIEW: gold_high_activity_insiders
    ✅ Created VIEW: 2 insider records


In [0]:
# VIEW 3: gold_combined_smart_money
# Combined view for smart money signals

try:
    print("\n📊 Creating VIEW: gold_combined_smart_money")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_combined_smart_money AS
    SELECT 
        t.company_name,
        t.insider_name,
        t.filing_date,
        t.transaction_date,
        t.acquired_disposed,
        t.shares,
        t.price_per_share,
        ROUND(CAST(t.shares AS DOUBLE) * CAST(t.price_per_share AS DOUBLE), 2) as transaction_value,
        t.confidence_score,
        CASE 
            WHEN CAST(t.confidence_score AS DOUBLE) >= 80 THEN 'STRONG'
            WHEN CAST(t.confidence_score AS DOUBLE) >= 60 THEN 'MODERATE'
            ELSE 'WEAK'
        END as signal_strength
    FROM {silver_schema}.silver_fact_insider_transactions t
    WHERE CAST(t.confidence_score AS DOUBLE) >= {min_conviction}
        AND t.filing_date >= '{lookback_start}'
    ORDER BY CAST(t.confidence_score AS DOUBLE) DESC, t.filing_date DESC
    """)
    
    signal_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_combined_smart_money").collect()[0]['count']
    print(f"    ✅ Created VIEW: {signal_count:,} signal records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise


📊 Creating VIEW: gold_combined_smart_money
    ✅ Created VIEW: 357 signal records


In [0]:
# COMMAND ----------
# VIEW 4: gold_insider_sentiment
# Sentiment analysis by company and date

try:
    print("\n📊 Creating VIEW: gold_insider_sentiment")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_insider_sentiment AS
    SELECT 
        company_name,
        filing_date,
        SUM(CASE WHEN acquired_disposed = 'A' THEN 1 ELSE 0 END) as buy_count,
        SUM(CASE WHEN acquired_disposed = 'D' THEN 1 ELSE 0 END) as sell_count,
        COUNT(*) as total_transactions,
        ROUND(
            100.0 * SUM(CASE WHEN acquired_disposed = 'A' THEN 1 ELSE 0 END) / COUNT(*), 
            2
        ) as buy_percentage,
        ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) as avg_confidence,
        CASE 
            WHEN SUM(CASE WHEN acquired_disposed = 'A' THEN 1 ELSE 0 END) > 
                 SUM(CASE WHEN acquired_disposed = 'D' THEN 1 ELSE 0 END) 
            THEN 'BULLISH'
            WHEN SUM(CASE WHEN acquired_disposed = 'A' THEN 1 ELSE 0 END) < 
                 SUM(CASE WHEN acquired_disposed = 'D' THEN 1 ELSE 0 END) 
            THEN 'BEARISH'
            ELSE 'NEUTRAL'
        END as sentiment
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    GROUP BY company_name, filing_date
    ORDER BY filing_date DESC, company_name
    """)
    
    sentiment_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_insider_sentiment").collect()[0]['count']
    print(f"    ✅ Created VIEW: {sentiment_count:,} sentiment records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise



📊 Creating VIEW: gold_insider_sentiment
    ✅ Created VIEW: 89 sentiment records


In [0]:
# VIEW 5: gold_institutional_conviction
# Conviction scores by institution and company

try:
    print("\n📊 Creating VIEW: gold_institutional_conviction")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_institutional_conviction AS
    SELECT 
        company_name,
        security_title,
        COUNT(*) as transaction_count,
        ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) as avg_conviction_score,
        MAX(CAST(confidence_score AS DOUBLE)) as max_conviction_score,
        MIN(CAST(confidence_score AS DOUBLE)) as min_conviction_score,
        MAX(filing_date) as latest_date,
        ROUND(
            (MAX(CAST(confidence_score AS DOUBLE)) - MIN(CAST(confidence_score AS DOUBLE))) 
            / NULLIF(MAX(CAST(confidence_score AS DOUBLE)), 0) * 100, 
            2
        ) as conviction_volatility
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    GROUP BY company_name, security_title
    ORDER BY avg_conviction_score DESC
    """)
    
    conviction_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_institutional_conviction").collect()[0]['count']
    print(f"    ✅ Created VIEW: {conviction_count:,} conviction records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise


📊 Creating VIEW: gold_institutional_conviction
    ✅ Created VIEW: 55 conviction records


In [0]:
# VIEW 6: gold_kpi_summary
# Key performance indicators

try:
    print("\n📊 Creating VIEW: gold_kpi_summary")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_kpi_summary AS
    SELECT 
        '{run_date}' as metric_date,
        'Total Transactions' as metric_name,
        CAST(COUNT(*) AS STRING) as metric_value,
        '{environment}' as environment
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    
    UNION ALL
    
    SELECT 
        '{run_date}' as metric_date,
        'Total Companies' as metric_name,
        CAST(COUNT(DISTINCT company_name) AS STRING) as metric_value,
        '{environment}' as environment
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    
    UNION ALL
    
    SELECT 
        '{run_date}' as metric_date,
        'Total Insiders' as metric_name,
        CAST(COUNT(DISTINCT insider_name) AS STRING) as metric_value,
        '{environment}' as environment
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    
    UNION ALL
    
    SELECT 
        '{run_date}' as metric_date,
        'Avg Confidence Score' as metric_name,
        CAST(ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) AS STRING) as metric_value,
        '{environment}' as environment
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    """)
    
    kpi_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_kpi_summary").collect()[0]['count']
    print(f"    ✅ Created VIEW: {kpi_count:,} KPI records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise



📊 Creating VIEW: gold_kpi_summary
    ✅ Created VIEW: 4 KPI records


In [0]:
# VIEW 7: gold_smart_money_signals
# Smart money buy/sell signals

try:
    print("\n📊 Creating VIEW: gold_smart_money_signals")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_smart_money_signals AS
    SELECT 
        company_name,
        insider_name,
        filing_date as signal_date,
        transaction_date,
        CASE WHEN acquired_disposed = 'A' THEN 'BUY' WHEN acquired_disposed = 'D' THEN 'SELL' ELSE NULL END as signal_type,
        shares,
        price_per_share,
        ROUND(CAST(shares AS DOUBLE) * CAST(price_per_share AS DOUBLE), 2) as transaction_value,
        confidence_score,
        CASE 
            WHEN CAST(confidence_score AS DOUBLE) >= 80 THEN 'STRONG'
            WHEN CAST(confidence_score AS DOUBLE) >= 60 THEN 'MODERATE'
            ELSE 'WEAK'
        END as signal_strength,
        security_title
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE CAST(confidence_score AS DOUBLE) >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    ORDER BY CAST(confidence_score AS DOUBLE) DESC, filing_date DESC
    """)
    
    signals_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_smart_money_signals").collect()[0]['count']
    print(f"    ✅ Created VIEW: {signals_count:,} signal records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise


📊 Creating VIEW: gold_smart_money_signals
    ✅ Created VIEW: 357 signal records


In [0]:
# VIEW 8: gold_top_holdings_by_institution
# Top holdings analysis

try:
    print("\n📊 Creating VIEW: gold_top_holdings_by_institution")
    
    spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_schema}.gold_top_holdings_by_institution AS
    SELECT 
        company_name,
        insider_name,
        security_title,
        COUNT(*) as holding_count,
        SUM(CAST(shares AS BIGINT)) as total_shares,
        ROUND(AVG(CAST(price_per_share AS DOUBLE)), 2) as avg_price,
        MAX(filing_date) as latest_update,
        ROUND(AVG(CAST(confidence_score AS DOUBLE)), 2) as avg_confidence
    FROM {silver_schema}.silver_fact_insider_transactions
    WHERE confidence_score >= {min_conviction}
        AND filing_date >= '{lookback_start}'
    GROUP BY company_name, insider_name, security_title
    """)
    
    holdings_count = spark.sql(f"SELECT COUNT(*) as count FROM {gold_schema}.gold_top_holdings_by_institution").collect()[0]['count']
    print(f"    ✅ Created VIEW: {holdings_count:,} holding records")
    
except Exception as e:
    print(f"    ❌ Error creating view: {str(e)}")
    raise


📊 Creating VIEW: gold_top_holdings_by_institution
    ✅ Created VIEW: 257 holding records


In [0]:
# ===================================================================
# STEP 3: Summary & Completion
# ===================================================================

print("\n" + "="*80)
print("🎉 GOLD LAYER VIEWS CREATED SUCCESSFULLY")
print("="*80)

print(f"""
Summary:
  ✅ gold_insider_summary_by_company
  ✅ gold_high_activity_insiders
  ✅ gold_combined_smart_money
  ✅ gold_insider_sentiment
  ✅ gold_institutional_conviction
  ✅ gold_kpi_summary
  ✅ gold_smart_money_signals
  ✅ gold_top_holdings_by_institution
  
  Environment: {environment}
  Run date: {run_date}
  All views created as logical views (not physical tables)
""")



🎉 GOLD LAYER VIEWS CREATED SUCCESSFULLY

Summary:
  ✅ gold_insider_summary_by_company
  ✅ gold_high_activity_insiders
  ✅ gold_combined_smart_money
  ✅ gold_insider_sentiment
  ✅ gold_institutional_conviction
  ✅ gold_kpi_summary
  ✅ gold_smart_money_signals
  ✅ gold_top_holdings_by_institution
  
  Environment: prod
  Run date: 2026-02-22
  All views created as logical views (not physical tables)



In [0]:
print("\n✅ Gold layer ready for dashboards and reporting!")

# Return success
dbutils.notebook.exit("SUCCESS")